In [82]:
# #load packeges
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import warnings
from scipy.stats import skew
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy.stats import norm
from collections import Counter
warnings.filterwarnings('ignore')
sns.set(style='white', context='notebook', palette='deep')
%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [85]:
# Load train and Test set
#train = pd.read_csv("data/housing_price_train.csv")
#test = pd.read_csv("data/housing_price_test.csv")

TRAIN_FILE = pd.read_csv("data/basketball.train.csv")
TEST_FILE = pd.read_csv("data/basketball.test.csv")
OUTPUT_FILE = "basketball2.result.csv"

In [59]:
print(TRAIN_FILE.size)
print(TEST_FILE.size)

965705
445692


In [12]:
TRAIN_FILE.head()

,H,W,13,3,1,399,11.1,0,3.7,0.5,2,1.1,missed
0,A,L,-12,6,2,369,6.8,0,3.2,16.3,2,5.9,made
1,A,L,-4,2,2,449,24.0,1,1.6,4.9,2,1.5,missed
2,A,W,12,7,3,436,16.1,0,1.1,20.2,2,5.0,missed
3,A,W,4,16,4,253,14.0,5,5.5,8.1,2,3.0,missed
4,A,L,-19,12,3,359,6.6,2,3.6,7.3,2,2.8,made


In [14]:
print(TRAIN_FILE.values)

[['A' 'L' -12 ... 2 5.9 'made']
 ['A' 'L' -4 ... 2 1.5 'missed']
 ['A' 'W' 12 ... 2 5.0 'missed']
 ...
 ['A' 'W' 8 ... 2 8.6 'made']
 ['A' 'L' -9 ... 3 7.2 'missed']
 ['A' 'L' -8 ... 2 9.6 'missed']]


In [29]:
TEST_FILE.head()

,H,W,12,16,3,161,11,5,4.6,6.5,2,2.3
0,H,W,4,3,1,343,15.3,0,0.7,3.1,2,2.7
1,A,L,-27,2,1,318,15.6,0,1.0,24.5,3,5.0
2,A,L,-6,5,2,253,16.7,7,7.9,3.0,2,1.1
3,A,W,2,4,1,365,12.8,1,2.8,4.3,2,2.3
4,H,L,-7,1,1,681,13.3,3,4.0,17.3,2,3.9


In [31]:
TEST_FILE.values

array([['H', 'W', 4, ..., 3.1, 2, 2.7],
       ['A', 'L', -27, ..., 24.5, 3, 5.0],
       ['A', 'L', -6, ..., 3.0, 2, 1.1],
       ...,
       ['A', 'W', 5, ..., 25.8, 3, 3.3],
       ['H', 'L', -2, ..., 22.2, 3, 5.9],
       ['A', 'L', -5, ..., 6.8, 2, 3.3]], dtype=object)

### Building tree
A trick to build an effective tree is to understand which questions to ask and when. And to do that we need to quantify how much a question helps to unmix the label. And we can quantify the amount of uncertainty at a single node using a metrik called Ginni impurity. And we can quantify how much a question reduces that uncertainty using a concept called Information Gain. We will uses this to select the best questions to ask at each point.
We will continue deviding the data until there are no more questions to ask at which point we will add a leaf.
We need to understand ehat type of questions to ask and how to decide which question to ask when

To generate a list of questions we will iterate through every value for every feature that apears in the dataset. Each of this becomes a candidate for a treshold that can be used to partition the data.

In response to a question we partition the data in two subsets, the first contains all the rowa for which the question is true and the second contains everythin else.

The best question is that reduces our uncertainty the most. And Gini impurity lets us quantify how much uncertainty the is at a node. Information Gain lets us quentify how much a question reduces that.
Gini impurity gives us our chances of bein incorect in case where we draw a record from a data and assign a random label to it.

Information Gain wil let us fingd a question that reduces out uncertainty the most and its just a number that tells us how much a queston helps to unmix the labels at a node.

In [32]:
from __future__ import print_function

train = TRAIN_FILE.values
test = TEST_FILE.values

In [61]:
len(test)

37141

In [33]:
header = ["LOCATION","W","FINAL_MARGIN", "SHOT_NUMBER", "PERIOD","GAME_CLOCK", "SHOT_CLOCK", "DRIBBLES","TOUCH_TIME", "SHOT_DIST", "PTS_TYPE", "CLOSE_DEF_DIST", "Target"]


In [34]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [35]:
def class_counts(rows):
    counts = {}  
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [36]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [37]:
class Question:

    def __init__(self, column, value):
        self.column = column  
        self.value = value    

    def match(self, example):
        val = example[self.column]  
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value 

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

In [38]:
def partition(rows, question):   # partition to two halfs one that matches the question and another that doesnt
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [39]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows)) #probabilty of a label
        impurity -= prob_of_lbl**2
    return impurity

In [40]:
# number that describes how much a question helps to unmix the label at a node
# the biggest value would be the best question to ask at a particular node
def info_gain(left, right, current_uncertainty):# the bigger infogain the better
    p = float(len(left)) / (len(left) + len(right))
    infogain = current_uncertainty - p * gini(left) - (1 - p) * gini(right)
    return infogain

In [41]:
def find_best_split(rows):
    best_gain = 0  
    best_question = None 
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  

    for col in range(n_features): 
        values = set([row[col] for row in rows])  
        for val in values:  
            question = Question(col, val)  
            true_rows, false_rows = partition(rows, question)
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

### Generating tree

In [42]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [43]:
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [44]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [45]:
my_tree = build_tree(train)

In [46]:
my_tree_cashe = my_tree

### Classifying

In [86]:
TEST = "data/basketball.test.csv"

In [87]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [88]:
def create_output(test, OUTPUT_FILE):
    temp_list = []
    with open(test) as f :
        csvreader = csv.reader(f,delimiter=',')
        cnt = 0
        for row in csvreader :
            cnt +=1
            key_value = classify(row, my_tree)
            temp = sorted(key_value.keys())[0]
            temp_list.append([cnt, temp])
    file = open(OUTPUT_FILE,"w")
    file.write("Id,Target\n") 
    for row in temp_list:
        file.write("%s,%s\n" % (row[0], row[1]))
    file.close() 


In [89]:
create_output(TEST, OUTPUT_FILE)